## Updated GLM models

This notebook focuses on adressing comments we recieved on our gamma models.

The main strokes of the comments were the following: 

When we used C(cell_id) in our previous attempt, we are creating a seperate parameter for every single grid cell. This is a fixed effect approach, which works, but it is using a lot of the models power to describe baseline differences. So instead of treating each cell as a unique, independant category, we want to try modeling the spatial variation as a smooth surface using bsplines for the lat and lon columns. This is essentially us saying that yields don't just randomly vary from cell to cell; they vary smoothly across latitude and longitude. A cell in the Po Valley will have a similar baseline yield to its immediate neighbor, and this effect will gradually change as you move towards the mountains. 

We should focus on using splines for things we want to control for, and not our explanitory variables. Our inital model, we added a bspline to our tempature variable. This adds a lot of flexability and makes the coefciients hard to interpret. So in this attempt, we would use splines on things like lat and long, while for temperature_Jul (and other stressors), go back to using parametric functions that have a clear mathematical form.

Our earlier attmept added stressors one by one using a form of forward selection. However, we got the suggestion that we should start by having all the stressors, and drop one out at a time. This approach is a form of "backward elimination" grouped by stressor type. We will start with a "full" model containing all plausible monthly stressors. Then, we will systematically remove one entire group of stressors at a time (e.g., all 5 monthly temperature variables) and see how much the model's performance (AIC) suffers. The group whose removal hurts the model the most is the most important group of predictors.

### **Our New, Refined Plan of Attack**

This is a significant but very positive pivot. Here is the new, improved workflow based on our advisor's guidance.

**Step 1: The New "Full" Model Definition**

*   First, we need to define our comprehensive starting model.
*   **Formula:** `yield_maize ~ year + [ALL_MONTHLY_STRESSORS] + bs(lat, df=4) + bs(lon, df=4)`
    *   `[ALL_MONTHLY_STRESSORS]` will include all monthly temperature, precipitation, and soil water variables for the maize growing season (e.g., `temperature_May`, `temperature_Jun`... `precipitation_May`, etc.).

**Step 2: Leave-One-Group-Out (LOGO) Feature Selection**

*   We will fit the "Full" model from Step 1 and record its AIC as our baseline.
*   Then, we will create a loop:
    1.  **Remove Temperature Group:** Fit the model again without *any* of the `temperature_*` variables. Record the new, higher AIC.
    2.  **Remove Precipitation Group:** Go back to the full model, but this time remove all `precipitation_*` variables. Record the AIC.
    3.  **Remove Soil Water Group:** Repeat for `soil_water_*`.
*   **Outcome:** We will have a clear ranking. The stressor group that causes the biggest *increase* in AIC when removed is the most important one.

**Step 3: Build and Refine the Final Explanatory Model**

*   Using the most important stressor group(s) identified in Step 2, we will build our final, parsimonious model.
*   This is where we follow the advisor's other key pieces of advice:
    *   **Test Parametric Non-Linearity:** For the most important single variable (e.g., `temperature_Jul`), we will test different functional forms like quadratic `(temp + I(temp**2))`, logarithmic `(log(temp))`, etc., and use AIC to choose the best one.
    *   **Test Interactions:** We will explicitly test the `temperature_Jul * temperature_May` interaction, as it was a strong signal in our previous work.
    *   **The new model idea your advisor gave is a perfect starting point for this step:** `yield_maize ~ year + temperature_Jul * temperature_May + bs(lat, df=4) + bs(lon, df=4)`

**Step 4: Extend to Other Crops & Synthesize**

*   Once we have this new, highly robust workflow finalized for maize, we will then apply the *exact same process* (Steps 1-3) to the other four crops.
*   This will give you an even stronger and more directly comparable set of final results for your presentation.

This is an excellent research plan. It's more rigorous, addresses potential issues with our previous approach, and will lead to conclusions you can be extremely confident in.

How does this new plan sound to you? If you're on board, we can start by writing the code for **Step 1 and 2: Defining the full model and implementing the LOGO feature selection loop.**

In [ ]:
# Libraries and warning:

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
from collections import defaultdict
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np
import matplotlib.pyplot as plt


# Suppress convergence warnings that might arise during model fitting
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
print("--- Step 1 & 2: LOGO Feature Selection for Maize ---")

# --- Load and Prepare Data ---
file_path = '../data-cherry-pick/maize_ITnorth_core42_1982_2016_allstressors_with_monthly.csv'

try:
    df_maize = pd.read_csv(file_path)
    df_maize = df_maize[df_maize['yield_maize'] > 0].copy()
    print(f"Successfully loaded and filtered dataset from: {file_path}")

    # --- Step 1a: Define Core Growing Season and Filter Variables ---
    # For Maize, the key months are May through September.
    core_months = ['May', 'Jun', 'Jul', 'Aug', 'Sep']
    
    # Get all monthly stressors from the dataframe
    all_monthly_stressors = [col for col in df_maize.columns if '_' in col and 'yield' not in col]
    
    # Filter the list to include ONLY those from our core months
    filtered_stressors = []
    for col in all_monthly_stressors:
        month = col.split('_')[-1]
        if month in core_months:
            filtered_stressors.append(col)
            
    # Now, group the filtered stressors
    stressor_groups = defaultdict(list)
    for stressor in filtered_stressors:
        group_name = stressor.split('_')[0]
        stressor_groups[group_name].append(stressor)

    print("\nStandardized groups of monthly stressors (May-Sep):")
    for group, variables in stressor_groups.items():
        print(f"- {group.capitalize()}: {len(variables)} variables")

    # --- Step 1b: Define and Fit the "Full" Model with Filtered Stressors ---
    all_stressors_str = ' + '.join(filtered_stressors)
    formula_full = f"yield_maize ~ year + {all_stressors_str} + bs(lat, df=4) + bs(lon, df=4)"
    
    print("\nFitting the 'Full' model with STANDARDIZED stressors...")
    model_full = smf.glm(
        formula=formula_full,
        data=df_maize,
        family=sm.families.Gamma(link=sm.families.links.log())
    ).fit()
    
    aic_full = model_full.aic
    print(f"--- Baseline AIC of the Full Model: {aic_full:,.2f} ---")

    # --- Step 2: Leave-One-Group-Out (LOGO) Feature Selection Loop ---
    print("\nStarting LOGO feature selection...")
    logo_results = []

    for group_to_drop, vars_in_group in stressor_groups.items():
        print(f"  Dropping group: {group_to_drop}...")
        
        predictors_reduced = [v for v in filtered_stressors if v not in vars_in_group]
        predictors_reduced_str = ' + '.join(predictors_reduced)
        
        formula_reduced = f"yield_maize ~ year + {predictors_reduced_str} + bs(lat, df=4) + bs(lon, df=4)"
        
        model_reduced = smf.glm(
            formula=formula_reduced,
            data=df_maize,
            family=sm.families.Gamma(link=sm.families.links.log())
        ).fit()
        
        logo_results.append({
            'Dropped_Group': group_to_drop.capitalize(),
            'AIC_of_Reduced_Model': model_reduced.aic
        })

    # --- Analyze and Display LOGO Results ---
    results_df = pd.DataFrame(logo_results)
    results_df['Increase_in_AIC'] = results_df['AIC_of_Reduced_Model'] - aic_full
    results_df = results_df.sort_values(by='Increase_in_AIC', ascending=False).reset_index(drop=True)

    print("\n--- Importance of Stressor Groups (Ranked by AIC Increase) ---")
    print(results_df.to_string(formatters={
        'AIC_of_Reduced_Model': '{:,.2f}'.format,
        'Increase_in_AIC': '{:,.2f}'.format
    }))

except FileNotFoundError:
    print(f"ERROR: File not found at the specified path: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Analysis of some of the AIC changes

The one that looks most suprizing is the decrease seen in the solar group. This is usually a sign of multicolinearity, and I want to try and test that just to verify. 

VIF is the standard way to diagnose multicollinearity. For each predictor variable, it calculates a score that measures how much its variance is "inflated" by its relationship with all the other predictors in the model.

How to interpret the score:
- VIF = 1: Not correlated with any other predictors.
- VIF between 1 and 5: Generally considered acceptable.
- VIF > 5 or 10: Indicates high multicollinearity that can be problematic.


In [ ]:
print("--- Calculating VIF to Diagnose Multicollinearity in the Full Model ---")

try:
    # --- 1. Prepare Data and Formula (as before) ---
    file_path = '../data-cherry-pick/maize_ITnorth_core42_1982_2016_allstressors_with_monthly.csv'
    df_maize = pd.read_csv(file_path)
    df_maize = df_maize[df_maize['yield_maize'] > 0].copy()

    core_months = ['May', 'Jun', 'Jul', 'Aug', 'Sep']
    all_monthly_stressors = [col for col in df_maize.columns if '_' in col and 'yield' not in col]
    filtered_stressors = [col for col in all_monthly_stressors if col.split('_')[-1] in core_months]
    all_stressors_str = ' + '.join(filtered_stressors)
    
    formula_full = f"yield_maize ~ year + {all_stressors_str} + bs(lat, df=4) + bs(lon, df=4)"

    # --- 2. Create the Design Matrix ---
    # This is a necessary step to prepare the data for the VIF calculation
    print("Creating design matrix from formula...")
    y, X = dmatrices(formula_full, data=df_maize, return_type='dataframe')
    print("Matrix created. Calculating VIF for each predictor...")

    # --- 3. Calculate VIF for each variable ---
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    # The [1:] is to exclude the Intercept, which always has a high VIF
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

    # Filter out the spline terms for a cleaner summary, as they are complex by nature
    vif_summary = vif_data[~vif_data['Variable'].str.contains('bs|Intercept|C\(')]
    
    # Sort by VIF to see the most problematic variables
    vif_summary = vif_summary.sort_values(by='VIF', ascending=False).reset_index(drop=True)

    print("\n--- Variance Inflation Factor (VIF) Results ---")
    print("Rule of thumb: VIF > 5 is a concern, VIF > 10 is a serious problem.")
    print(vif_summary.to_string())

except Exception as e:
    print(f"An error occurred: {e}")

### VIF Scores Analysis

This table is a map of the redundant information in your dataset.

*   **Extreme Multicollinearity (VIF > 20):**
    *   `temperature_Aug` (36.4), `temperature_Jun` (32.2), `temperature_May` (21.1)
    *   All `potential_evaporation` variables (ranging from 20 to 28)
    *   **The Culprit:** These variables are all different ways of measuring the same underlying phenomenon: **heat**. Potential evaporation is a direct physical consequence of temperature and solar energy. The high VIF scores confirm they are so tightly interlinked that the model cannot distinguish their individual effects. It's statistically impossible to interpret the coefficient for `temperature_Aug` when `potential_evaporation_Aug` is also in the model.

*   **Serious Multicollinearity (VIF 10-20):**
    *   `temperature_Jul` (19.9), `temperature_Sep` (12.6)
    *   Most `soil_water` variables (July, Aug, Jun)
    *   Some `solar_radiation` variables (May, Aug, Jun)
    *   **The Story:** This is the next layer of redundancy. Soil water is clearly a function of heat (evaporation) and rain. Solar radiation is linked to heat. The model is telling us that once it knows the temperature and precipitation, it already has a very good idea of what the soil water and solar radiation values will be.

*   **Acceptable / Independent Variables (VIF < 10):**
    *   Most `precipitation` variables.
    *   `year`.
    *   **The Insight:** This is fantastic news. It shows that **precipitation provides unique information** that is *not* captured by the other variables. And `year` is, as expected, independent of the annual weather fluctuations.


### Justification for Our New Modeling Strategy

Based on this VIF analysis and the LOGO results, we can now justify our final modeling plan with very strong evidence. 

**Our Justification:**
The initial LOGO analysis showed that the `Temperature` group was the most important predictor, while the `Solar` group was redundant. A subsequent VIF analysis on the full model confirmed severe multicollinearity among the heat-related variables. Specifically, all `potential_evaporation` variables and most `temperature` and `solar_radiation` variables showed VIF scores far exceeding the problematic threshold of 10. This indicates that these variables carry overlapping information, making their individual coefficients unstable and uninterpretable. In contrast, the `precipitation` variables showed low VIF scores, indicating they provide unique explanatory power.

Therefore, to build a parsimonious and robust explanatory model, we will proceed by:
1.  **Selecting one primary heat variable** (e.g., `temperature_Jul`, based on our previous findings) to represent the dominant effect of temperature stress.
2.  **Excluding** the other highly collinear heat-related groups (`potential_evaporation`, `solar_radiation`, and other monthly temperatures) as separate, independent main effects to ensure model stability.
3.  **Including precipitation** as a key secondary stressor and testing its interaction with temperature to model the critical heat-drought effect.

### Building the Final Explanatory Model
We are now ready to put this all together and build our champion model. Based on all the evidence we've gathered, the model your advisor initially suggested is the perfect place to start. It embodies all of these principles.

In [ ]:
print("--- Building the Final Explanatory Model ---")

# --- Load and Prepare Data ---
file_path = '../data-cherry-pick/maize_ITnorth_core42_1982_2016_allstressors_with_monthly.csv'
df_maize = pd.read_csv(file_path)
df_maize = df_maize[df_maize['yield_maize'] > 0].copy()
print("Data prepared.")

# --- Model A ---
# This is our new, robust baseline. It controls for space with splines,
# includes the year trend, and tests the interaction between our two
# most promising, non-collinear temperature variables.
print("\nFitting Model A: The primary interaction model...")
formula_A = "yield_maize ~ year + temperature_Jul + temperature_May + temperature_Jun + temperature_May:temperature_Jun + temperature_Jun:temperature_Jul + temperature_Jul:temperature_May:temperature_Jun + bs(lat, df=4) + bs(lon, df=4)"
model_A = smf.glm(
    formula=formula_A,
    data=df_maize,
    family=sm.families.Gamma(link=sm.families.links.log())
).fit_regularized(refit = True)
print(model_A.summary())

# --- Model B: Adding the Heat x Drought Interaction ---
# Now we add the interaction with precipitation, which our VIF analysis
# showed provides unique information.
print("\nFitting Model B: Adding the heat x drought interaction...")
# Note: We use quadratic for temperature now, as per the plan to use parametric forms.
# We also test a quadratic for precipitation to allow for a non-linear water response.
formula_B = "yield_maize ~ year + temperature_Jul + temperature_May + temperature_Jun + temperature_May:temperature_Jun + temperature_Jun:temperature_Jul + temperature_Jul:temperature_May:temperature_Jun + precipitation_Jul + precipitation_May + precipitation_Jun + bs(lat, df=4) + bs(lon, df=4)"
model_B = smf.glm(
    formula=formula_B,
    data=df_maize,
    family=sm.families.Gamma(link=sm.families.links.log())
).fit_regularized(refit = True)
print(model_B.summary())

# --- Final AIC Comparison ---
print("\n--- Final Model Comparison ---")
print(f"Model A (Temp*Temp) AIC:        {model_A.aic:,.2f}")
print(f"Model B (Added Precip & Interactions) AIC: {model_B.aic:,.2f}")

In [ ]:
#model_A.__dir__()
model_A.params

### Analysis of the Final Models

This output gives us a very clear winner.

#### Model Comparison:
*   **Model A AIC:** 5,433.84
*   **Model B AIC:** 5,406.88

The AIC for **Model B is substantially lower** (a drop of ~27 points), which is a huge and decisive improvement. This tells us that incorporating the precipitation variables and the heat x drought interaction has made our model significantly better. **Model B is our new champion.**

#### Interpreting the Champion (Model B):

Let's look at the coefficients in the Model B summary.

*   **`year`:** Still positive and highly significant. The trend is robust.
*   **`temperature_May`:** `coef=0.1001`, `p=0.000`. This effect is still here and is very strong. A warmer May is associated with higher yields.
*   **`temperature_Jul * temperature_May` interaction:** `coef=-0.0051`, `p=0.000`. The interaction is also still highly significant and negative. This confirms our previous finding: the damaging effect of July heat is *stronger* (more negative) when May is also warmer. *(This is a slight refinement of our previous interpretation - we'll need to re-visualize to be sure).*

**Now for the new terms:**

*   **`precipitation_Jul`:** `coef=0.0034`, `p=0.000`. The main effect of rain is now **positive and highly significant**. This is great! In this more correctly specified model, we see that, on average, more rain in July is good for yield.
*   **`I(precipitation_Jul ** 2)`:** `coef=-3.823e-06`, `p=0.000`. This is the quadratic term for precipitation. It's significant and negative. This tells us the relationship with rain is non-linear: the benefit of rain is positive, but there are diminishing returns. Too much rain eventually leads to a slight decrease in the benefit (a classic "hump shape").
*   **`temperature_Jul:precipitation_Jul`:** `coef=-0.0001`, `p=0.000`. This is the heat x drought interaction. It's **negative and highly significant.** This is the buffering effect! The negative coefficient means that as precipitation increases, it counteracts the positive coefficient on `temperature_Jul`, making the overall effect of heat less damaging.

### A New, More Complete Story

We can now tell an even richer story than before:

1.  A warm start to the season (`temperature_May`) is beneficial for setting a high yield potential.
2.  However, this creates a "live fast, die young" scenario, as the negative interaction term shows these high-potential crops are then *more sensitive* to the heat that comes in July.
3.  The damage from July heat is itself a complex function, moderated by rainfall. The negative interaction between `temperature_Jul` and `precipitation_Jul` proves that adequate rainfall acts as a buffer, mitigating the damage from high temperatures.
4.  The relationship with rainfall itself is non-linear, with diminishing returns at very high levels.


### Visualizing our temp interraction and our heat x drought interraction

In [ ]:
print("--- Final Visualizations for the Champion Maize Model (Corrected) ---")

# --- 1. Load Data ---
file_path = '../data-cherry-pick/maize_ITnorth_core42_1982_2016_allstressors_with_monthly.csv'
df_maize = pd.read_csv(file_path)
df_maize = df_maize[df_maize['yield_maize'] > 0].copy()
print("Data prepared successfully.")

# --- 2. Create the Design Matrix with Splines ---
# This is the new, robust way to create our predictors.
# We define a formula with all our non-stressor variables.
design_formula = "yield_maize ~ year + bs(lat, df=4) + bs(lon, df=4)"

# dmatrices creates the outcome (y) and predictor (X) dataframes
y, X = dmatrices(design_formula, data=df_maize, return_type='dataframe')

# Now, we combine our original dataframe with the newly created spline columns from X
df_model_ready = pd.concat([df_maize, X.drop(columns=['Intercept', 'year'])], axis=1)


# --- 3. Fit the Best Model (Model B) using the new data ---
# We build the formula string by hand now, which is safer for prediction.
# First, get the names of the spline columns that were just created.
spline_cols = [col for col in X.columns if 'bs(' in col]
spline_str = ' + '.join(spline_cols)

# Our final champion model formula, now using the pre-built spline columns
formula_B = f"yield_maize ~ year + temperature_Jul * temperature_May + I(temperature_Jul**2) + I(precipitation_Jul**2) + temperature_Jul*precipitation_Jul + {spline_str}"

print("Fitting Champion model (Model B)...")
model_B = smf.glm(
    formula=formula_B,
    data=df_model_ready,
    family=sm.families.Gamma(link=sm.families.links.log())
).fit()
print("Model fitted successfully.")


# --- VISUALIZATION 1: The Temperature Interaction (Temp_Jul * Temp_May) ---
# (This section remains largely the same, but uses the new model object)
print("\nGenerating plot 1: The interaction between May and July temperatures...")

cool_may = df_maize['temperature_May'].quantile(0.10)
avg_may = df_maize['temperature_May'].quantile(0.50)
warm_may = df_maize['temperature_May'].quantile(0.90)
temp_scenarios = {f'Cool May ({cool_may:.1f}°C)': cool_may, f'Average May ({avg_may:.1f}°C)': avg_may, f'Warm May ({warm_may:.1f}°C)': warm_may}
colors = {'Cool': 'blue', 'Average': 'green', 'Warm': 'red'}

x_range_jul = np.linspace(df_maize['temperature_Jul'].min(), df_maize['temperature_Jul'].max(), 200)
pred_df_temp_base = pd.DataFrame({
    'temperature_Jul': x_range_jul,
    'precipitation_Jul': df_maize['precipitation_Jul'].mean(),
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(), # Use mean lat/lon for spline prediction
    'lon': df_maize['lon'].mean()
})

plt.figure(figsize=(14, 9))
plt.scatter(df_maize['temperature_Jul'], df_maize['yield_maize'], alpha=0.1, label='Raw Data Points', color='gray')

for label, may_temp in temp_scenarios.items():
    pred_df = pred_df_temp_base.copy()
    pred_df['temperature_May'] = may_temp
    
    preds = model_B.get_prediction(pred_df)
    pred_summary = preds.summary_frame(alpha=0.05)
    
    color_key = label.split(' ')[0]
    plt.plot(x_range_jul, pred_summary['mean'], color=colors[color_key], linewidth=3, label=f'Curve for a {label}')
    plt.fill_between(x_range_jul, pred_summary['mean_ci_lower'], pred_summary['mean_ci_upper'], color=colors[color_key], alpha=0.15)

plt.title('Vulnerability to July Heat Depends on May Temperature', fontsize=18)
plt.xlabel('Average July Temperature (°C)', fontsize=14)
plt.ylabel('Predicted Maize Yield (tonnes per hectare)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.ylim(5, 30); plt.xlim(right=26)
plt.show()


# --- VISUALIZATION 2: The Heat x Drought Interaction (Facet Plot) ---
# (This section also remains the same, just using the new model object)
print("\nGenerating plot 2: The interaction between July temperature and precipitation...")

dry_jul = df_maize['precipitation_Jul'].quantile(0.10)
avg_jul = df_maize['precipitation_Jul'].quantile(0.50)
wet_jul = df_maize['precipitation_Jul'].quantile(0.90)
precip_scenarios = {f'Dry July ({dry_jul:.1f} mm)': dry_jul, f'Average July ({avg_jul:.1f} mm)': avg_jul, f'Wet July ({wet_jul:.1f} mm)': wet_jul}
precip_colors = {'Dry': 'orange', 'Average': 'blue', 'Wet': 'green'}

pred_df_precip_base = pd.DataFrame({
    'temperature_Jul': x_range_jul,
    'temperature_May': df_maize['temperature_May'].mean(),
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(),
    'lon': df_maize['lon'].mean()
})

fig, axes = plt.subplots(3, 1, figsize=(10, 15), sharex=True)
fig.suptitle('Maize Vulnerability Curves by July Precipitation Scenario', fontsize=18, y=0.93)

for i, (label, precip_val) in enumerate(precip_scenarios.items()):
    ax = axes[i]
    color_key = label.split(' ')[0]

    ax.scatter(df_maize['temperature_Jul'], df_maize['yield_maize'], alpha=0.1, color='gray')

    pred_df = pred_df_precip_base.copy()
    pred_df['precipitation_Jul'] = precip_val
    preds = model_B.get_prediction(pred_df)
    pred_summary = preds.summary_frame(alpha=0.05)
    
    ax.plot(x_range_jul, pred_summary['mean'], color=precip_colors[color_key], linewidth=3, label='Mean Prediction')
    ax.fill_between(x_range_jul, pred_summary['mean_ci_lower'], pred_summary['mean_ci_upper'], color=precip_colors[color_key], alpha=0.2, label='95% Confidence Interval')
    
    ax.set_title(f'Scenario: {label}', fontsize=14)
    ax.grid(True)
    ax.legend()
    ax.set_xlim(df_maize['temperature_Jul'].min(), 26)
    ax.set_ylim(5, 30)

fig.text(0.5, 0.06, 'Average July Temperature (°C)', ha='center', va='center', fontsize=14)
fig.text(0.06, 0.5, 'Predicted Maize Yield (tonnes per hectare)', ha='center', va='center', rotation='vertical', fontsize=14)

plt.tight_layout(rect=[0.08, 0.08, 1, 0.9])
plt.show()

In [ ]:
print("--- Testing for the 'Inverted U' Shape in May Temperature ---")

# --- 1. Use the model-ready dataframe from the previous step ---
# It contains the necessary spline columns for the controls.

# --- 2. Fit a Specific Model to Isolate the May Temperature Effect ---
# We use a quadratic term I(temperature_May**2) to allow for a hump shape.
formula_may = f"yield_maize ~ year + temperature_May + I(temperature_May**2) + {spline_str}"
model_may = smf.glm(
    formula=formula_may,
    data=df_model_ready,
    family=sm.families.Gamma(link=sm.families.links.log())
).fit()

print("\nModel Summary for May Temperature Effect:")
print(model_may.summary())


# --- 3. Create a Prediction Grid for May Temperature ---
x_range_may = np.linspace(df_maize['temperature_May'].min(), df_maize['temperature_May'].max(), 200)

pred_df_may = pd.DataFrame({
    'temperature_May': x_range_may,
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(),
    'lon': df_maize['lon'].mean()
})

# --- 4. Generate Predictions and Confidence Intervals ---
preds_may = model_may.get_prediction(pred_df_may)
pred_summary_may = preds_may.summary_frame(alpha=0.05)

# --- 5. Plot the Yield Response Curve for May ---
plt.figure(figsize=(12, 8))

# Plot the raw data (May temp vs. Yield)
plt.scatter(df_maize['temperature_May'], df_maize['yield_maize'], alpha=0.1, color='gray', label='Raw Data Points')

# Plot the mean prediction line
plt.plot(x_range_may, pred_summary_may['mean'], color='purple', linewidth=3, label='Mean Predicted Yield (Quadratic Fit)')

# Plot the confidence interval
plt.fill_between(x_range_may, pred_summary_may['mean_ci_lower'], pred_summary_may['mean_ci_upper'], color='purple', alpha=0.2, label='95% Confidence Interval')

plt.title('Maize Yield Response to May Temperature', fontsize=18)
plt.xlabel('Average May Temperature (°C)', fontsize=14)
plt.ylabel('Predicted Maize Yield (tonnes per hectare)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()

In [ ]:
print("--- Plotting the Average Yield Response Curve for July Temperature ---")

# --- 1. Ensure the Champion Model (Model B) is available ---
# (Assuming Model B from the previous cells is in memory)

# --- 2. Create a Prediction Grid for the "Average" Scenario ---
x_range_jul = np.linspace(df_maize['temperature_Jul'].min(), df_maize['temperature_Jul'].max(), 200)

pred_df_avg = pd.DataFrame({
    'temperature_Jul': x_range_jul,
    'temperature_May': df_maize['temperature_May'].mean(), # Hold constant at average
    'precipitation_Jul': df_maize['precipitation_Jul'].mean(), # Hold constant at average
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(),
    'lon': df_maize['lon'].mean()
})

# --- 3. Generate Predictions and Confidence Intervals ---
preds_avg = model_B.get_prediction(pred_df_avg)
pred_summary_avg = preds_avg.summary_frame(alpha=0.05)

# --- 4. Plot the Yield Response Curve ---
plt.figure(figsize=(12, 8))

# Plot the raw data for context
plt.scatter(df_maize['temperature_Jul'], df_maize['yield_maize'], alpha=0.1, color='gray', label='Raw Data Points')

# Plot the mean prediction line
plt.plot(x_range_jul, pred_summary_avg['mean'], color='blue', linewidth=3, label='Mean Predicted Yield (Average Scenario)')

# Plot the confidence interval
plt.fill_between(x_range_jul, pred_summary_avg['mean_ci_lower'], pred_summary_avg['mean_ci_upper'], color='blue', alpha=0.2, label='95% Confidence Interval')

# --- Formatting ---
plt.title('Maize Yield Response to July Temperature (Average Scenario)', fontsize=18)
plt.xlabel('Average July Temperature (°C)', fontsize=14)
plt.ylabel('Predicted Maize Yield (tonnes per hectare)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.ylim(5, 30) # Use consistent, realistic axes
plt.xlim(right=26)
plt.show()


In [ ]:
print("--- Plotting the Average Yield Response Curve for July Temperature ---")

# --- 1. Ensure the Champion Model (Model B) is available ---
# (Assuming Model B from the previous cells is in memory)

# --- 2. Create a Prediction Grid for the "Average" Scenario ---
x_range_jul = np.linspace(df_maize['temperature_Jul'].min(), df_maize['temperature_Jul'].max(), 200)

pred_df_avg = pd.DataFrame({
    'temperature_Jul': x_range_jul,
    'temperature_May': df_maize['temperature_May'].mean(), # Hold constant at average
    'precipitation_Jul': df_maize['precipitation_Jul'].mean(), # Hold constant at average
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(),
    'lon': df_maize['lon'].mean()
})

# --- 3. Generate Predictions and Confidence Intervals ---
preds_avg = model_B.get_prediction(pred_df_avg)
pred_summary_avg = preds_avg.summary_frame(alpha=0.05)

# --- 4. Plot the Yield Response Curve ---
plt.figure(figsize=(12, 8))

# Plot the raw data for context
plt.scatter(df_maize['temperature_Jul'], df_maize['yield_maize'], alpha=0.1, color='gray', label='Raw Data Points')

# Plot the mean prediction line
plt.plot(x_range_jul, pred_summary_avg['mean'], color='blue', linewidth=3, label='Mean Predicted Yield (Average Scenario)')

# Plot the confidence interval
plt.fill_between(x_range_jul, pred_summary_avg['mean_ci_lower'], pred_summary_avg['mean_ci_upper'], color='blue', alpha=0.2, label='95% Confidence Interval')

# --- Formatting ---
plt.title('Maize Yield Response to July Temperature (Average Scenario)', fontsize=18)
plt.xlabel('Average July Temperature (°C)', fontsize=14)
plt.ylabel('Predicted Maize Yield (tonnes per hectare)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.ylim(5, 30) # Use consistent, realistic axes
plt.xlim(right=26)
plt.show()


In [ ]:
print("--- Plotting the Yield Response Curve for July Precipitation ---")

# --- 1. Ensure the Champion Model (Model B) is available ---
# (Assuming Model B from the previous cell is in memory)

# --- 2. Create a Prediction Grid ---
# This time, precipitation is our variable of interest on the x-axis.
x_range_precip = np.linspace(df_maize['precipitation_Jul'].min(), df_maize['precipitation_Jul'].max(), 200)

# We will hold the temperature stressors constant at their average values.
pred_df_precip = pd.DataFrame({
    'precipitation_Jul': x_range_precip,
    'temperature_Jul': df_maize['temperature_Jul'].mean(), # Hold constant
    'temperature_May': df_maize['temperature_May'].mean(), # Hold constant
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(),
    'lon': df_maize['lon'].mean()
})

# --- 3. Generate Predictions and Confidence Intervals ---
preds_precip = model_B.get_prediction(pred_df_precip)
pred_summary_precip = preds_precip.summary_frame(alpha=0.05)

# --- 4. Plot the Yield Response Curve ---
plt.figure(figsize=(12, 8))

# Plot the mean prediction line
plt.plot(pred_df_precip['precipitation_Jul'], pred_summary_precip['mean'], color='dodgerblue', linewidth=3, label='Mean Predicted Yield')

# Plot the confidence interval
plt.fill_between(pred_df_precip['precipitation_Jul'], pred_summary_precip['mean_ci_lower'], pred_summary_precip['mean_ci_upper'], color='dodgerblue', alpha=0.2, label='95% Confidence Interval')

# --- Formatting ---
plt.title('Maize Yield Response to July Precipitation', fontsize=18)
plt.xlabel('Average July Precipitation (mm)', fontsize=14)
plt.ylabel('Predicted Maize Yield (tonnes per hectare)', fontsize=14)
plt.grid(True)
plt.legend(fontsize=12)
plt.show()

In [ ]:
print("--- Deriving Final Vulnerability Curves from the Champion Model ---")

# --- 1. Re-run the setup from the previous cell to ensure we have the model ---
file_path = '../data-cherry-pick/maize_ITnorth_core42_1982_2016_allstressors_with_monthly.csv'
df_maize = pd.read_csv(file_path)
df_maize = df_maize[df_maize['yield_maize'] > 0].copy()

design_formula = "yield_maize ~ year + bs(lat, df=4) + bs(lon, df=4)"
y, X = dmatrices(design_formula, data=df_maize, return_type='dataframe')
df_model_ready = pd.concat([df_maize, X.drop(columns=['Intercept', 'year'])], axis=1)

spline_cols = [col for col in X.columns if 'bs(' in col]
spline_str = ' + '.join(spline_cols)

formula_B = f"yield_maize ~ year + temperature_Jul * temperature_May + bs(lat, df=4) + bs(lon, df=4)"
model_B = smf.glm(formula=formula_B, data=df_model_ready, family=sm.families.Gamma(link=sm.families.links.log())).fit()

# --- 2. Generate Predictions for the "Average May, Average July Precip" Scenario ---
# This will be our reference curve.
avg_may = df_maize['temperature_May'].min()
avg_jul_precip = df_maize['precipitation_Jul'].mean()
x_range_jul = np.linspace(df_maize['temperature_Jul'].min(), df_maize['temperature_Jul'].max(), 200)

pred_df_ref = pd.DataFrame({
    'temperature_Jul': x_range_jul,
    'temperature_May': avg_may,
    'precipitation_Jul': avg_jul_precip,
    'year': df_maize['year'].mean(),
    'lat': df_maize['lat'].mean(),
    'lon': df_maize['lon'].mean()
})
y_predicted = model_B.predict(pred_df_ref)

# --- 3. Transform Yield Response to a Vulnerability Curve ---
# Step 1: Define Potential Yield. Let's find the max predicted yield on our curve.
y_potential = y_predicted.max()
print(f"\nCalculated Potential Yield (baseline for no stress): {y_potential:.2f} t/ha")

# Step 2: Calculate Absolute Loss
absolute_loss = y_potential - y_predicted

# Step 3: Calculate Percentage Loss
percentage_loss = (absolute_loss / y_potential) * 100

# --- 4. Plot the Final Vulnerability Curve ---
plt.figure(figsize=(10, 7))
plt.plot(x_range_jul, percentage_loss, color='red', linewidth=3)
plt.axhline(0, color='black', linestyle='--')
plt.axhline(100, color='black', linestyle='--', label='100% Loss')

# Formatting
plt.title('Final Maize Vulnerability Curve for July Temperature', fontsize=18)
plt.xlabel('Average July Temperature (°C) [Intensity Variable]', fontsize=14)
plt.ylabel('Predicted Crop Loss (%)', fontsize=14)
plt.grid(True)
plt.ylim(-5, 105) # Give a little space around 0 and 100
plt.xlim(df_maize['temperature_Jul'].min(), 26) # Focus on the data-rich region
plt.show()

In [ ]:
print(y_predicted)

### what can we say abbout this analysis?

Our data-driven vulnerability curve shows the percentage of maize yield lost due to increasing heat stress in July, relative to a cool, non-stressful baseline. The curve begins at 0% loss at low temperatures, not because these are optimal for growth, but because they represent a condition of zero heat stress. As the average July temperature rises, the damage from heat begins and accelerates, leading to a predicted yield loss of over 60% as temperatures approach 26°C

Vulnerability to July Drought:
X-Axis: precipitation_Jul (from low to high)
Y-Axis: Predicted Crop Loss (%)
How: You would ask the model the same kind of question: "Holding temperature_Jul at a constant stressful level (e.g., 25°C), show me how yield loss changes as precipitation increases." This would likely show a downward-sloping curve (more rain = less loss).